In [1]:
import arcpy
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# show all columns
pd.options.display.max_columns = None

In [5]:
# store directories
inputs = '.\\Outputs\\saltlake.gdb'
outputs = '.\\Post_Processing'

In [6]:
# store residential dataset paths
sf = os.path.join(inputs, '_02_single_family')
sf_adu = os.path.join(inputs, '_02_single_family_adu')
duplex = os.path.join(inputs, '_02_duplex')
pud = os.path.join(inputs, '_02_pud')
condo = os.path.join(inputs, '_02_condo')
mobile_home_parks = os.path.join(inputs, '_02_mobile_home_park')
low_income_housing = os.path.join(inputs, '_02_LowIncomeApts')
apartments = os.path.join(inputs, '_02_apartment')

In [7]:
# update the subtype_wfrc field
arcpy.CalculateField_management(sf, field='SUBTYPE_WFRC', expression="'single_family'", expression_type="PYTHON3")
arcpy.CalculateField_management(sf_adu, field='SUBTYPE_WFRC', expression="'single_family'", expression_type="PYTHON3")
arcpy.CalculateField_management(duplex, field='SUBTYPE_WFRC', expression="'duplex'", expression_type="PYTHON3")
arcpy.CalculateField_management(apartments, field='SUBTYPE_WFRC', expression="'apartment'", expression_type="PYTHON3")
arcpy.CalculateField_management(low_income_housing, field='SUBTYPE_WFRC', expression="'apartment_low_income'", expression_type="PYTHON3")
arcpy.CalculateField_management(mobile_home_parks, field='SUBTYPE_WFRC', expression="'mobile_home'", expression_type="PYTHON3")

<Result '.\\Outputs\\saltlake.gdb\\_02_mobile_home_park'>

#### load feature classes into spatial dataframes

In [8]:
sf_df = pd.DataFrame.spatial.from_featureclass(sf)
duplex_df = pd.DataFrame.spatial.from_featureclass(duplex)
sf_adu_df = pd.DataFrame.spatial.from_featureclass(sf_adu)

mobile_home_parks_df = pd.DataFrame.spatial.from_featureclass(mobile_home_parks)
low_income_housing_df = pd.DataFrame.spatial.from_featureclass(low_income_housing)
apartments_df = pd.DataFrame.spatial.from_featureclass(apartments)

pud_df = pd.DataFrame.spatial.from_featureclass(pud)
condo_df = pd.DataFrame.spatial.from_featureclass(condo)

In [9]:
# Create crosswalk dictionary
crosswalk = {'CO':'condo', 'SF':'single_family', 'TH':'townhouse', 'DP':'duplex', 'MX':'mixed'}
pud_df['SUBTYPE_WFRC'] = pud_df['SUBTYPE_WFRC'].map(crosswalk)
condo_df['SUBTYPE_WFRC'] = condo_df['SUBTYPE_WFRC'].map(crosswalk)

#### format tables so that they are stackable

In [10]:
# sf
sf_df = sf_df[['TOTAL_MKT_VALUE', 'LAND_MKT_VALUE', 'PARCEL_ACRES', 'HOUSE_CNT', 'BLDG_SQFT', 'BUILT_YR', 'EFFBUILT_YR', 
               'TYPE_WFRC', 'SUBTYPE_WFRC' ]].copy()
sf_df['ap_count'] = 1
sf_df['parcel_count'] = 1


# sf_adu
sf_adu_df = sf_adu_df[['TOTAL_MKT_VALUE', 'LAND_MKT_VALUE', 'PARCEL_ACRES', 'HOUSE_CNT', 'BLDG_SQFT', 'BUILT_YR', 'EFFBUILT_YR', 
                       'TYPE_WFRC', 'SUBTYPE_WFRC' ]].copy()
sf_adu_df['ap_count'] = 1
sf_adu_df['parcel_count'] = 1


# duplex
duplex_df = duplex_df[['TOTAL_MKT_VALUE', 'LAND_MKT_VALUE', 'PARCEL_ACRES', 'HOUSE_CNT', 'BLDG_SQFT', 'BUILT_YR', 'EFFBUILT_YR', 
                       'TYPE_WFRC', 'SUBTYPE_WFRC' ]].copy()
duplex_df['ap_count'] = 2
duplex_df['parcel_count'] = 1

# mobile_home_parks
mobile_home_parks_df = mobile_home_parks_df[['TOTAL_MKT_VALUE', 'LAND_MKT_VALUE', 'PARCEL_ACRES', 'HOUSE_CNT', 'BLDG_SQFT', 
                                             'BUILT_YR', 'EFFBUILT_YR','TYPE_WFRC', 'SUBTYPE_WFRC', 
                                             'ap_count' ]].copy()
mobile_home_parks_df['parcel_count'] = 0 # should run spatial join to get actual parcel count later

# low_income_housing
low_income_housing_df = low_income_housing_df[['TOTAL_MKT_VALUE', 'LAND_MKT_VALUE', 'PARCEL_ACRES', 'HOUSE_CNT', 'BLDG_SQFT', 
                                               'BUILT_YR', 'EFFBUILT_YR', 'TYPE_WFRC', 'SUBTYPE_WFRC', 
                                               'ap_count' ]].copy()
low_income_housing_df['parcel_count'] = 0 # should run spatial join to get actual parcel count later

# apartments
apartments_df = apartments_df[['TOTAL_MKT_VALUE', 'LAND_MKT_VALUE', 'PARCEL_ACRES', 'HOUSE_CNT', 'BLDG_SQFT', 'BUILT_YR', 
                               'EFFBUILT_YR','TYPE_WFRC', 'SUBTYPE_WFRC', 'ap_count' ]].copy()
apartments_df['parcel_count'] = 0 # should run spatial join to get actual parcel count later

# pud
pud_df = pud_df[['TOTAL_MKT_VALUE', 'LAND_MKT_VALUE', 'PARCEL_ACRES', 'HOUSE_CNT', 'BLDG_SQFT', 'BUILT_YR', 'EFFBUILT_YR', 
               'TYPE_WFRC', 'SUBTYPE_WFRC', 'ap_count','parcel_count']].copy()
# condo
condo_df = condo_df[['TOTAL_MKT_VALUE', 'LAND_MKT_VALUE', 'PARCEL_ACRES', 'HOUSE_CNT', 'BLDG_SQFT', 'BUILT_YR', 'EFFBUILT_YR', 
               'TYPE_WFRC', 'SUBTYPE_WFRC', 'ap_count','parcel_count']].copy()

#### stack em

In [12]:
frames = [sf_df, sf_adu_df, duplex_df, mobile_home_parks_df, low_income_housing_df, apartments_df, pud_df, condo_df]
residential = pd.concat(frames)

# calculate the decade
residential['BUILT_DECADE'] = 'NA'
residential.loc[(residential['BUILT_YR'] >= 1840) & (residential['BUILT_YR'] < 1850), 'BUILT_DECADE'] = "1840's"
residential.loc[(residential['BUILT_YR'] >= 1850) & (residential['BUILT_YR'] < 1860), 'BUILT_DECADE'] = "1850's"
residential.loc[(residential['BUILT_YR'] >= 1860) & (residential['BUILT_YR'] < 1870), 'BUILT_DECADE'] = "1860's"
residential.loc[(residential['BUILT_YR'] >= 1870) & (residential['BUILT_YR'] < 1880), 'BUILT_DECADE'] = "1870's"
residential.loc[(residential['BUILT_YR'] >= 1880) & (residential['BUILT_YR'] < 1890), 'BUILT_DECADE'] = "1880's"
residential.loc[(residential['BUILT_YR'] >= 1890) & (residential['BUILT_YR'] < 1900), 'BUILT_DECADE'] = "1890's"
residential.loc[(residential['BUILT_YR'] >= 1900) & (residential['BUILT_YR'] < 1910), 'BUILT_DECADE'] = "1900's"
residential.loc[(residential['BUILT_YR'] >= 1910) & (residential['BUILT_YR'] < 1920), 'BUILT_DECADE'] = "1910's"
residential.loc[(residential['BUILT_YR'] >= 1920) & (residential['BUILT_YR'] < 1930), 'BUILT_DECADE'] = "1920's"
residential.loc[(residential['BUILT_YR'] >= 1930) & (residential['BUILT_YR'] < 1940), 'BUILT_DECADE'] = "1930's"
residential.loc[(residential['BUILT_YR'] >= 1940) & (residential['BUILT_YR'] < 1950), 'BUILT_DECADE'] = "1940's"
residential.loc[(residential['BUILT_YR'] >= 1950) & (residential['BUILT_YR'] < 1960), 'BUILT_DECADE'] = "1950's"
residential.loc[(residential['BUILT_YR'] >= 1960) & (residential['BUILT_YR'] < 1970), 'BUILT_DECADE'] = "1960's"
residential.loc[(residential['BUILT_YR'] >= 1970) & (residential['BUILT_YR'] < 1980), 'BUILT_DECADE'] = "1970's"
residential.loc[(residential['BUILT_YR'] >= 1980) & (residential['BUILT_YR'] < 1990), 'BUILT_DECADE'] = "1980's"
residential.loc[(residential['BUILT_YR'] >= 1990) & (residential['BUILT_YR'] < 2000), 'BUILT_DECADE'] = "1990's"
residential.loc[(residential['BUILT_YR'] >= 2000) & (residential['BUILT_YR'] < 2010), 'BUILT_DECADE'] = "2000's"
residential.loc[(residential['BUILT_YR'] >= 2010) & (residential['BUILT_YR'] < 2020), 'BUILT_DECADE'] = "2010's"
residential.loc[(residential['BUILT_YR'] >= 2020) & (residential['BUILT_YR'] < 2030), 'BUILT_DECADE'] = "2020's"

# export 
residential.to_csv(os.path.join(outputs, 'salt_lake_county_residential_units.csv'))